In [1]:
import numpy as np
from numpy import random

class Teachers:
    def __init__(self):
        self.Tname = np.array([
                                ["Pumipong"],  #0
                                ["Atikhom"],   #1
                                ["Kittikorn"], #2
                                ["Mayoon"],    #3
                                ["Thanawut"]   #4
                              ])
    
    def getTeacherName(self):
        return self.Tname
    
class Rooms:
    def __init__(self):        
        self.Rname = np.array( [
                                ["4911"], #0
                                ["4912"], #1
                                ["4914"], #2
                                ["4921"], #3
                                ["4922"]  #4
                             ] )
        
        self.SlotRoom = np.array( [
                                    ["xxxxxxx"],["xxxxxxx"], #0
                                    ["xxxxxxx"],["xxxxxxx"], #1
                                    ["xxxxxxx"],["xxxxxxx"], #2
                                    ["xxxxxxx"],["xxxxxxx"], #3
                                    ["xxxxxxx"],["xxxxxxx"]  #4
                                ] )
        self.SlotRoomWeek = np.array( [self.SlotRoom for _ in range(len(self.Rname))] # Ex . for five rooms
                                    )
    def getRoomsName(self):
        return self.Rname
    
    def getRoomWeek(self):
        return self.SlotRoomWeek

class CoursesWorkload:
    def __init__(self):
        self.CWL= np.array([   # Course , Teacher , Room, TimeSlot 
                                ["CPE1111", 0, -1, -1 ], #0
                                ["CPE1112", 0, -1, -1 ], #1
                                ["CPE1113", 0, -1, -1 ], #2
                                ["CPE2111", 0, -1, -1 ], #3
                                ["CPE2112", 0, -1, -1 ], #4
                                ["CPE2113", 0, -1, -1 ], #5
                                ["CPE2114", 0, -1, -1 ], #6
                                ["CPE2115", 0, -1, -1 ], #7
                                ["CPE2100", 4, -1, -1 ], #8
                                ["CPE2101", 4, -1, -1 ]  #9
                            ])
    def getCourseWorkload(self):
        return self.CWL

In [ ]:
# ------ Main Class ------
def Scheduling(Cwl, Rm):
    for i in range(len(Cwl)):
        rdRoom = random.randint(0,len(Rm)-1)
        rdSlot = random.randint(0,len(Rm[0])-1)
        while (Rm[rdRoom, rdSlot] != "xxxxxxx"):
            rdRoom = random.randint(0,len(Rm)-1)
            rdSlot = random.randint(0,len(Rm[0])-1)
        Cwl[i,2] = rdRoom    #Cwl[i,2] = random.randint(0,len(Rm)-1)
        Cwl[i,3] = rdSlot    #Cwl[i,3] = random.randint(0,len(Rm[0])-1)
        Rm[int(Cwl[i,2]), int(Cwl[i,3])] = Cwl[i,0]  #Rm[int(Cwl[i,2]), int(Cwl[i,3])] = Cwl[i,0]

def ComputeFittnessValue(Cwl, Rm) :
    value = 50
    for r in range(len(Rm)):
        if (Rm[r,6] != "xxxxxxx") :
            value -= 1
        if (Rm[r,7] != "xxxxxxx") :
            value -= 1
            
    for i in range(0,len(Cwl)) :
        for j in range(i,len(Cwl)) :
            if (i != j ) :
                if (Cwl[i,1] == Cwl[j,1] and Cwl[i,3] == Cwl[j,3]) :
                    value = 0
                if (Cwl[i,2] == Cwl[j,2] and Cwl[i,3] == Cwl[j,3]) :
                    value = 0
    return value

def DisplayRoom(R):
    Rm = R.getRoomWeek()
    for i in range(len(Rm)):
        print()
        print(R.Rname[i])
        for sl in range(len(Rm[i])):
            print(Rm[i,sl], end=" ")
            if sl%2 == 1 :
                print()

def DisplayCourseWorkLoad(cwl):
    print(cwl)

#np.random.seed(0) 
fitness = 0
while (fitness != 50) :
    T = Teachers()
    R = Rooms()
    C = CoursesWorkload()
    Scheduling(C.getCourseWorkload(), R.getRoomWeek())
    fitness = ComputeFittnessValue(C.getCourseWorkload(), R.getRoomWeek())
    
    if (fitness == 50) :
        DisplayRoom(R)
        print(fitness)
        DisplayCourseWorkLoad(C.getCourseWorkload())